In [103]:
import numpy as np
import time
from sklearn.metrics import accuracy_score
import scipy.optimize as optim
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import math
from tabulate import tabulate
from sklearn.model_selection import KFold, cross_val_score
from numpy import mean, std
import warnings
warnings.filterwarnings("ignore")

## Data Edit & Clean

In [104]:
# load data
data=pd.read_csv('/Users/zhangyudan/Documents/Github/fall2022-project4-proj4-group2-main/data/compas-scores-two-years.csv')
data.shape

(7214, 53)

In [105]:
#since we are only intersted in sample fairness between two races: African-American and Caucasian
#and LFR needs to cageorize sensitive group and nonsensitive group,we regard defandant with African-American as 0,Caucasian as 1
data = data[(data['race']=='African-American') | (data['race']=='Caucasian')]

In [106]:
# find the number of nan for each column
nan_counts = data.isna().sum()
print(nan_counts.sort_values())

id                            0
end                           0
start                         0
priors_count.1                0
v_screening_date              0
v_score_text                  0
v_decile_score                0
v_type_of_assessment          0
screening_date                0
score_text                    0
decile_score.1                0
type_of_assessment            0
is_violent_recid              0
event                         0
is_recid                      0
c_charge_degree               0
two_year_recid                0
juv_fel_count                 0
sex                           0
compas_screening_date         0
age                           0
decile_score                  0
last                          0
juv_other_count               0
dob                           0
first                         0
age_cat                       0
name                          0
race                          0
priors_count                  0
juv_misd_count                0
c_days_f

In [107]:
# drop columns that having too many Nan
data = data.dropna(thresh = len(data) - 21, axis = 1)
# drop unnecessary columns 
data = data.drop(columns = ['id', 'name', 'first', 'last','compas_screening_date','c_case_number','dob','age','screening_date','v_screening_date','type_of_assessment','v_type_of_assessment'])
# drop rows with Nan 
data = data.dropna()
# The recidivist flag (is_recid) should be -1 if we could not find a compas case at all.
data = data.loc[data['is_recid'] != -1]
# Ordinary traffic offenses (c_charge_degree = 'O') will not result in Jail time and hence are removed 
data = data.loc[data['c_charge_degree'] != 'O']
# score_text shouldn't be 'N/A'
data = data.loc[data['score_text'] != 'N/A']
# convert Series to int
data.c_charge_desc = pd.to_numeric(data.c_charge_desc, errors='coerce').fillna(0, downcast='infer')


In [108]:
unique_count = data.nunique()
print(unique_count.sort_values())

c_charge_desc            1
is_violent_recid         2
is_recid                 2
event                    2
c_charge_degree          2
sex                      2
race                     2
two_year_recid           2
age_cat                  3
score_text               3
v_score_text             3
juv_other_count          9
decile_score            10
juv_fel_count           10
decile_score.1          10
v_decile_score          10
juv_misd_count          10
priors_count            37
priors_count.1          37
start                  221
c_days_from_compas     454
end                   1091
dtype: int64


- Before building the LFR model, we need to transfer variables to dummy variables .
- Split data into Sensitive and Nonsensitive data

In [109]:
# replacing values
data['sex'].replace(['Male','Female'],[1,0], inplace = True)
data['race'].replace(['Caucasian','African-American'],[1,0], inplace = True)
data['c_charge_degree'].replace(['M','F'],[1,0], inplace = True)

data['age_cat'].replace(['Less than 25','25 - 45','Greater than 45'],['A','B','C'], inplace = True)
data.loc[data['age_cat']=='A', 'age_cat1'] = 1
data.loc[data['age_cat']!='A', 'age_cat1'] = 0
data.loc[data['age_cat']=='B', 'age_cat2'] = 1
data.loc[data['age_cat']!='B', 'age_cat2'] = 0

data['v_score_text'].replace(['Low','Medium','High'],['C','B','A'], inplace = True)
data.loc[data['v_score_text']=='A', 'v_score_text1'] = 1
data.loc[data['v_score_text']!='A', 'v_score_text1'] = 0
data.loc[data['v_score_text']=='B', 'v_score_text2'] = 1
data.loc[data['v_score_text']!='B', 'v_score_text2'] = 0

data['score_text'].replace(['Low','Medium','High'],['B','C','A'], inplace = True)
data.loc[data['score_text']=='A', 'score_text1'] = 1
data.loc[data['score_text']!='A', 'score_text1'] = 0
data.loc[data['score_text']=='B', 'score_text2'] = 1
data.loc[data['score_text']!='B', 'score_text2'] = 0

# drop original categorical variables
data=data.drop(columns=['age_cat','v_score_text','score_text'])

In [110]:
data.shape

(6129, 25)

In [111]:
data

,sex,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,c_days_from_compas,c_charge_degree,c_charge_desc,...,start,end,event,two_year_recid,age_cat1,age_cat2,v_score_text1,v_score_text2,score_text1,score_text2
1,1,0,0,3,0,0,0,1.0,0,0,...,9,159,1,1,0.0,1.0,0.0,0.0,0.0,1.0
2,1,0,0,4,0,1,4,1.0,0,0,...,0,63,0,1,1.0,0.0,0.0,0.0,0.0,1.0
3,1,0,0,8,1,0,1,1.0,0,0,...,0,1174,0,0,1.0,0.0,0.0,1.0,1.0,0.0
6,1,1,0,6,0,0,14,1.0,0,0,...,5,40,1,1,0.0,1.0,0.0,0.0,0.0,0.0
8,0,1,0,1,0,0,0,1.0,1,0,...,2,747,0,0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7207,1,0,0,2,0,0,0,1.0,1,0,...,0,529,1,1,0.0,1.0,0.0,0.0,0.0,1.0
7208,1,0,0,9,0,0,0,1.0,0,0,...,0,169,0,0,1.0,0.0,1.0,0.0,1.0,0.0
7209,1,0,0,7,0,0,0,1.0,0,0,...,1,860,0,0,1.0,0.0,0.0,1.0,0.0,0.0
7210,1,0,0,3,0,0,0,1.0,0,0,...,1,790,0,0,1.0,0.0,0.0,1.0,0.0,1.0


## Data Splitting

Split the dataset into training, validation and testing set (0.6, 0.2, 0.2)

In [112]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=1)
data_train, data_val= train_test_split(data_train, test_size=0.25, random_state=1)

In [113]:
data_train=pd.concat([data_train.sex,data_train.race,data_train.juv_fel_count,data_train.decile_score,data_train.two_year_recid],axis=1)
data_train=data_train.iloc[:20]
data_val=pd.concat([data_val.sex,data_val.race,data_val.juv_fel_count,data_val.decile_score,data_val.two_year_recid],axis=1)
data_test=pd.concat([data_test.sex,data_test.race,data_test.juv_fel_count,data_test.decile_score,data_test.two_year_recid],axis=1)

In [114]:
data_train.head()

,sex,race,juv_fel_count,decile_score,two_year_recid
4178,1,0,0,7,0
2,1,0,0,4,1
126,1,1,0,6,1
6827,0,1,0,6,0
4662,1,1,0,1,1


# Learning Fair Representations

$d(x_n, v_k, \alpha) = \sum^D_{d=1} \alpha_d (x_{nd} - v_{kd})^2$

In [115]:
# distance - d(x_n, v_k, alpha)
def distance(X, v, alpha):
    N = X.shape[0]
    D = X.shape[1]
    K = len(v)

    
    res = np.zeros((N, K))
    
    for n in range(N):
        for k in range(K):
            for d in range(D):
                res[n, k] += alpha[d]*(X.iloc[n][d] - v[k, d])**2
    
    return res

$M_{nk} = P(Z=k|x_n) \space\space \forall n, k \\ 
= exp(-d(x_n, v_k))/\sum_{k=1}^K exp(-d(x_n, v_k))$

In [116]:
# M_nk = P(Z=k|x_n)is the probablity that x_n maps to v_k

def M_nk(dist, k):
    
    N = dist.shape[0]
    K = dist.shape[1]
    M_nk = np.zeros((N, K))
    expo_res = np.zeros((N, K))
    
    for n in range(N):
        deno = 0
        for k in range(K):
            expo_res[n, k] = math.exp((-1)*dist[n, k])
            deno += expo_res[n, k]
        for k in range(K):
            M_nk[n, k] = expo_res[n, k] / deno
    return M_nk

$M_k = \frac{1}{|X_0|} \sum_{n \in X_0} M_{nk}$

$X_0 = \{X_0^+, X_0^-\}$

$X_0$ is training data

In [117]:
#  M_k
def M_k(X, M_nk, k):
    N = X.shape[0]
    K = M_nk.shape[1]
    M_k = np.zeros(K)
    
    for k in range(K):
        for n in range(N):
            M_k[k] += M_nk[n, k]
        M_k[k] = M_k[k]/N
    return M_k

$
\hat{x}_n = \sum^K_{k=1}M_{nk}v_k
$

$
L_x = \sum_{n=1}^N (x_n - \hat{x}_n)^2
$

In [118]:
#  the reconstruction of x_n  and L_x
def x_n_hat(X, M_nk, v):

    N = M_nk.shape[0]
    D = X.shape[1]
    K = M_nk.shape[1]
    x_n_hat = np.zeros((N, D))
    L_x = 0
    
    
    for n in range(N):
        for d in range(D):
            for k in range(K):
                x_n_hat[n, d] += M_nk[n, k]*v[k, d]
        # calculate L_x        
        L_x += (X.iloc[n][d] - x_n_hat[n, d])**2
    
    return x_n_hat, L_x

$
L_y = \sum_{n=1}^N -y_n log \hat{y}_n - (1-y_n)log(1- \hat{y}_n)
$


minimize $L = A_z L_Z + A_x L_x + A_y L_y$ 

$A_x, A_z, A_y$ are hyperparameters governing trade-off between the system desiderata

In [119]:
# prediction for y_n and L_y
def y_n_hat(M_nk, w, y):
    N = M_nk.shape[0]
    K = M_nk.shape[1]
    y_n_hat = np.zeros(N)
    L_y = 0
    

    for n in range(N):
        for k in range(K):
            y_n_hat[n] += M_nk[n, k]*w[k]
        # calculate L_y
        L_y += (-1)*y.iloc[n]*np.log(y_n_hat[n]) - (1 - y.iloc[n])*np.log(1 - y_n_hat[n])
        
    return y_n_hat, L_y

In [120]:
#  the metric function we wanna minimize
def L(param, sen_df, nsen_df, sen_y, nonsen_y, K, A_z, A_x, A_y):
    # param is the list of parameters
    # sen_df is the sensitive dataset
    # nsen_df is the nonsensitive dataset
    # sen_y is the list of labels for sensitive dataset
    # nonsen_y is the list of labels for nonsensitive dataset
    # K, A_z, A_x and A_y are hyperparameters, the values are decided by the users
    
    sen_N, sen_D = sen_df.shape
    nsen_N, nsen_D = nsen_df.shape

   
    alpha_sen = param[:sen_D]
    alpha_nsen = param[sen_D : 2 * sen_D]
    w = param[2 * sen_D : (2 * sen_D) + K]
    v = np.matrix(param[(2 * sen_D) + K:]).reshape((K, sen_D))

    #  the distance matrix
    dist_sen = distance(sen_df, v, alpha_sen)
    dist_nsen = distance(nsen_df, v, alpha_nsen)        

    # M_nk matrix
    M_nk_sen = M_nk(dist_sen, K)
    M_nk_nsen = M_nk(dist_nsen, K)

    #  M_k matrix
    M_k_sen = M_k(sen_df, M_nk_sen, K)
    M_k_nsen = M_k(nsen_df, M_nk_nsen, K)

    #  L_z
    L_z = 0
    for k in range(K):
        L_z += abs(M_k_sen[k] - M_k_nsen[k])

    #  x_n_hat and L_x
    x_n_hat_sen, L_x_sen = x_n_hat(sen_df, M_nk_sen, v)
    x_n_hat_nsen, L_x_nsen = x_n_hat(nsen_df, M_nk_nsen, v)
    L_x = L_x_sen + L_x_nsen

    # y_n_hat and L_y
    y_hat_sen, L_y_sen = y_n_hat(M_nk_sen, w, sen_y)
    y_hat_nsen, L_y_nsen = y_n_hat(M_nk_nsen, w, nonsen_y)
    L_y = L_y_sen + L_y_nsen

    # the  metric function
    metric = A_z*L_z + A_x*L_x + A_y*L_y

    return metric

$
\hat{y}_n = \sum^K_{k=1} M_{nk}w_k \\
0< w_k <1
$

In [121]:
# defines the threshold for y_n_hat to be 0 or 1
def predic_threshold(preds):
    for i in range(len(preds)):
        if preds[i] >= 0.5:
            preds[i] = 1
        else:
            preds[i] = 0
    return preds

In [122]:
#  calculate y_n_hat by using the best parameters
def cal_pred(params, D, K, sen_dt, nsen_dt, sen_label, nsen_label):
    # form parameters in new forms
    best_alpha_sen = params[:D]
    best_alpha_nsen = params[D : 2 * D]
    best_w = params[2 * D : (2 * D) + K]
    best_v = np.matrix(params[(2 * D) + K:]).reshape((K, D))
    
    # calculate the distance matrix
    best_dist_sen = distance(sen_dt, best_v, best_alpha_sen)
    best_dist_nsen = distance(nsen_dt, best_v, best_alpha_nsen) 
    
    # calculate the M_nk matrix
    best_M_nk_sen = M_nk(best_dist_sen, K)
    best_M_nk_nsen = M_nk(best_dist_nsen, K)
    
    # calculate the y_n_hat matrix
    y_hat_sen, L_y_sen = y_n_hat(best_M_nk_sen, best_w, sen_label)
    y_hat_nsen, L_y_nsen = y_n_hat(best_M_nk_nsen, best_w, nsen_label)
    
    return y_hat_sen, y_hat_nsen

In [123]:
# calculates the  overall accuracy, 
# accuracy for  African-American group
# accuracy for  Caucasian group
# and calibration of the model
def cal_calibr(y_pred_sen, y_pred_nsen, y_sen_label, y_nsen_label):
    converted_y_hat_sen = predic_threshold(y_pred_sen)
    converted_y_hat_nsen = predic_threshold(y_pred_nsen)

    y_pred_sen = pd.DataFrame(converted_y_hat_sen)
    y_pred_nsen = pd.DataFrame(converted_y_hat_nsen)
     
    # calculate the accuracy
    acc_sen = accuracy_score(y_sen_label, y_pred_sen)
    acc_nsen = accuracy_score(y_nsen_label, y_pred_nsen)
    
    all_labels = y_sen_label.append(y_nsen_label)
    all_preds = y_pred_sen.append(y_pred_nsen)
    total_accuracy = accuracy_score(all_preds, all_labels)
    
    print("The overall accuracy  is: ", total_accuracy)
    print("The accuracy for African-American  is: ", acc_sen)
    print("The accuracy for Caucasian  is: ", acc_nsen)
    print("The calibration of the model is: ", abs(acc_sen-acc_nsen))

In [124]:
# LFR function 
#get train /validation accuracy
def LFR(training_data, val_data, y_name, sen_variable_name, K, A_z, A_x, A_y):
    # dividing the training dataset into sensitive and nonsensitive group
    sen_training = training_data[training_data[sen_variable_name]==0]
    nsen_training = training_data[training_data[sen_variable_name]==1]
    
    # dividing the validation dataset 
    sen_val = val_data[val_data[sen_variable_name]==0]
    nsen_val = val_data[val_data[sen_variable_name]==1]
    
    # delete sensitive variable 
    sen_training=sen_training.drop(columns=[sen_variable_name])
    sen_val=sen_val.drop(columns=[sen_variable_name])  
    nsen_training = nsen_training.drop(columns=[sen_variable_name])
    nsen_val = nsen_val.drop(columns=[sen_variable_name])
    
    # assign y labels 
    y_sen_training = sen_training[y_name]
    sen_training = sen_training.drop(columns=[y_name])
    y_sen_val = sen_val[y_name]
    sen_val = sen_val.drop(columns=[y_name])
    y_nsen_training = nsen_training[y_name]
    nsen_training = nsen_training.drop(columns=[y_name])
    y_nsen_val = nsen_val[y_name]
    nsen_val = nsen_val.drop(columns=[y_name])
    
    # pick random value
    # talpha and w  are between 0 and 1 and sum up to 1
    alpha_sen_1=np.random.random_sample((sen_training.shape[1],))
    alpha_nsen_1=np.random.random_sample((nsen_training.shape[1],))
    alpha_sen=alpha_sen_1/sum(alpha_sen_1)
    alpha_nsen=alpha_nsen_1/sum(alpha_nsen_1)
    w_1=np.random.random_sample((K,))
    w=w_1/sum(w_1)
    v=np.random.random((K, sen_training.shape[1]))
    
    
    initial = []
    initial.extend(alpha_sen)
    initial.extend(alpha_nsen)
    initial.extend(w)
    
    for item in v:
        initial.extend(item)
    initial = np.array(initial)

    # the boundary of the parameters
    bound=[]

    #  alpha and w are between 0 and 1 and sum up to 1
    for d in range(sen_training.shape[1]):
        bound.append((0, 1))

    for d in range(nsen_training.shape[1]):
        bound.append((0, 1))

    for k in range(K):
        bound.append((0, 1))

    
    for k in range(K):
        for d in range(sen_training.shape[1]):
            bound.append((None, None))
    
    # minimize the metric 
    para, min_L, d = optim.fmin_l_bfgs_b(L, x0=initial, epsilon=1e-5, 
                                         args=(sen_training, nsen_training, y_sen_training, 
                                               y_nsen_training, K, A_z, A_x, A_y), 
                                         bounds = bound, approx_grad=True, 
                                         maxfun=150000, maxiter=150000)
    
    # predict y_n_hat for the training dataset 
    y_hat_sen_tr, y_hat_nsen_tr = cal_pred(para, sen_training.shape[1], K, sen_training, 
             nsen_training, y_sen_training, y_nsen_training)

    print(" training accuracy:")
    cal_calibr(y_hat_sen_tr, y_hat_nsen_tr, y_sen_training, y_nsen_training)
    
    
    
    
    # predict y_n_hat for the validation dataset 
    y_hat_sen_val, y_hat_nsen_val = cal_pred(para, sen_val.shape[1], K, sen_val, 
             nsen_val, y_sen_val, y_nsen_val)  
    print("validation accuracy:")
    cal_calibr(y_hat_sen_val, y_hat_nsen_val, y_sen_val, y_nsen_val)    
    
    return  para

In [125]:
start = time.time()
para_test = LFR(data_train, data_val, 'two_year_recid', 'race', 10, 0.3, 0.3, 0.4)
end = time.time() 
print( f"The training time: {end-start}")

 training accuracy:
The overall accuracy  is:  0.8
The accuracy for African-American  is:  0.8181818181818182
The accuracy for Caucasian  is:  0.7777777777777778
The calibration of the model is:  0.04040404040404044
validation accuracy:
The overall accuracy  is:  0.5652528548123981
The accuracy for African-American  is:  0.6215083798882681
The accuracy for Caucasian  is:  0.48627450980392156
The calibration of the model is:  0.13523387008434656
The training time: 702.9030220508575


In [126]:
# Testing the LFR model
sen_test = data_test[data_test['race']==0]
nsen_test = data_test[data_test['race']==1]

sen_test=sen_test.drop(columns=['race'])
nsen_test = nsen_test.drop(columns=['race'])

y_sen_test = sen_test['two_year_recid']
sen_test = sen_test.drop(columns=['two_year_recid'])

y_nsen_test = nsen_test['two_year_recid']
nsen_test = nsen_test.drop(columns=['two_year_recid'])

start = time.time()
y_hat_sen_test, y_hat_nsen_test = cal_pred(para_test, sen_test.shape[1], 10, sen_test, 
             nsen_test, y_sen_test, y_nsen_test)
end = time.time() 
print( f"runtime of testing LFR model: {end-start}")
cal_calibr(y_hat_sen_test, y_hat_nsen_test, y_sen_test, y_nsen_test)

runtime of testing LFR model: 2.754948854446411
The overall accuracy  is:  0.5603588907014682
The accuracy for African-American  is:  0.6031537450722734
The accuracy for Caucasian  is:  0.49032258064516127
The calibration of the model is:  0.11283116442711211
